In [1]:
import sys
sys.path.append('./Library/') 
import get_data as gd
import pandas as pd

In [2]:
ods = gd.Get_Data(
    'postgres',
    'HRODS',
    'DEI')

In [3]:
engine = ods.set_engine()

#set paths
ods.set_output_path("C:\\Users\\garciand\\OneDrive - HP Inc\\Desktop\\Deliverables\\DEI\\")

In [4]:
query = """
SELECT 
		WD."Report Date",
		COALESCE(F1."Business Lvl 1 (Group) Code",WD."Business Lvl 1 (Group) Code") AS "Hybrid L1",
		COALESCE(F2."Business Lvl 2 (Unit) Code",WD."Business Lvl 2 (Unit) Code") AS "Hybrid L2",

		CASE
			WHEN WD."Work Address - City" = N'Pantnagar' THEN WD."Work Address - City"
			WHEN WD."Work Address - City" = N'Boise' THEN WD."Work Address - City"
			WHEN WD."Work Address - City" = N'Vancouver' THEN WD."Work Address - City"
			ELSE 'Other'
		END AS "Work Address - City",

		CASE
			WHEN TT."Job Family Code" IS NULL THEN 'No'
			ELSE 'Yes'
		END AS "Technical Job Family",

		CASE
			WHEN WD."Business Lvl 4 (MRU) Code" = N'G034' THEN WD."Business Lvl 4 (MRU) Code"
			ELSE 'Other'
		END AS "Business Lvl 4 (MRU) Code",
		CASE
			WHEN VET."Employee ID" IS NULL THEN 'U'
			ELSE VET."Veteran Status"
		END AS "Veteran Status",
		CASE
			WHEN WD."Pay Group Country Desc" IN (
			'Austria',
			'Belgium',
			'Bulgaria',
			'Croatia',
			'Czechia',
			'Denmark',
			'FinlAnd',
			'Greece',
			'Hungary',
			'IrelAnd',
			'Israel',
			'Kazakhstan',
			'Luxembourg',
			'Morocco',
			'NetherlAnds',
			'Nigeria',
			'Norway',
			'PolAnd',
			'Portugal',
			'Russian FederatiOn',
			'Saudi Arabia',
			'Serbia',
			'Slovakia',
			'South Africa',
			'Sweden',
			'Tunisia',
			'Turkey',
			'United Arab Emirates')
			THEN 'U'
			WHEN PWD."Employee ID" IS NULL THEN 'N'
			ELSE PWD."Is PWD?"
		END AS "PWD Status",

		CASE
			WHEN WD."Pay Group Country Code" = N'USA' THEN WD."Pay Group Country Desc"
			ELSE 'Other'
		END AS "Pay Group Country Desc",

		CASE
			WHEN WD."Pay Group Country Code" = N'USA' THEN WD."Ethnicity Group"
			ELSE 'Other'
		END AS "Ethnicity Group",
		COALESCE(WD."Gender Code",'U') AS "Gender Code",

		CASE
			WHEN WD."Management Level Category" <> 'NONE' THEN 'Yes'
			ELSE 'No'
		END AS "Is Professional",

		CASE
			WHEN WD."Original Hire Date" > '2020-10-31' OR WD."Hire Date" > '2020-10-31' THEN 'Yes'
			ELSE 'No'
		END AS "Is New Hire",
		WD."Management Level Category",

		CASE
			WHEN WD."Worker Reg / Temp Code" = 'R' AND WD."Worker Status Category Code" = 'A' THEN 1
			ELSE 0
		END "Headcount",
		CASE
			WHEN WD."Worker Status Category Code" = 'T' AND WD."Attrition Type" = 'Voluntary' THEN 1
			ELSE 0
		END "Voluntary Attrition",
		'HC' AS "Type"
	
	FROM "HPW_DATA" AS WD

	LEFT JOIN "JOB_FUNCTION" AS JF ON JF."Job Family Group" = WD."Job Family Group"
	LEFT JOIN "LABOR_PYRAMID" AS LP ON LP."Management Level" = WD."Management Level"
	LEFT JOIN "FEDL1" AS F1 ON F1."Business Lvl 4 (MRU) Code" = WD."Business Lvl 4 (MRU) Code"
	LEFT JOIN "FEDL2" AS F2 ON F2."Business Lvl 4 (MRU) Code" = WD."Business Lvl 4 (MRU) Code"
	LEFT JOIN "FEDL3" AS F3 ON F3."Business Lvl 4 (MRU) Code" = WD."Business Lvl 4 (MRU) Code"
	LEFT JOIN "TECHNICAL_JOBS" AS TT ON TT."Job Family Code" = WD."Job Family Code"
	LEFT JOIN "HPW_VETERANS" AS VET ON VET."Report Date" = WD."Report Date" AND VET."Employee ID" = WD."Worker ID" AND VET."Veteran Status" = 'Y'
	LEFT JOIN "HPW_W_DISABILITIES" AS PWD ON PWD."Employee ID" = WD."Worker ID" AND PWD."Report Date" = (Select Max("Report Date") From "HPW_W_DISABILITIES")

	WHERE 
		WD."Report Date" = (Select Max("Report Date") From "HPW_DATA")
		AND WD."Worker Reg / Temp Code" = 'R'
		AND WD."Worker Status Category Code" = 'A'
"""

In [5]:
data = ods.internal_query(query,engine)

In [6]:
# total woman in leadership US and global
ww_women_lead = data.loc[(data["Gender Code"] == "F")
    & (data["Management Level Category"].isin(['EXEC','DIR']))][
    "Headcount"
].sum()

us_lead_w = data.loc[
    (data["Pay Group Country Desc"] == "United States of America")
    & (data["Gender Code"] == "F")
    & (data["Management Level Category"].isin(['EXEC','DIR']))
]["Headcount"].sum()

ww_lead = data.loc[data["Management Level Category"].isin(['EXEC','DIR'])][
    "Headcount"
].sum()
# total woman in leadership ethnic vs white
us_eth_lead = data.loc[
    (data["Pay Group Country Desc"] == "United States of America")
    & (data["Gender Code"] == "F")
    & (data["Management Level Category"].isin(['EXEC','DIR']))
    & (data["Ethnicity Group"] != "WHITE_USA")
    & (~data["Ethnicity Group"].isin(['NSPEC_USA','UNK']))
]["Headcount"].sum()

us_white_lead = data.loc[
    (data["Pay Group Country Desc"] == "United States of America")
    & (data["Gender Code"] == "F")
    & (data["Management Level Category"].isin(['EXEC','DIR']))
    & (data["Ethnicity Group"] == "WHITE_USA")
]["Headcount"].sum()

us_lead = data.loc[
    (data["Pay Group Country Desc"] == "United States of America")
    & (data["Management Level Category"].isin(['EXEC','DIR']))
]["Headcount"].sum()

# total woman in Tech ethnic vs white
us_eth_tech = data.loc[
    (data["Pay Group Country Desc"] == "United States of America")
    & (data["Gender Code"] == "F")
    & (data["Technical Job Family"] == "Yes")
    & (data["Ethnicity Group"] != "WHITE_USA")
    & (~data["Ethnicity Group"].isin(['NSPEC_USA','UNK']))
]["Headcount"].sum()

us_white_tech = data.loc[
    (data["Pay Group Country Desc"] == "United States of America")
    & (data["Gender Code"] == "F")
    & (data["Technical Job Family"] == "Yes")
    & (data["Ethnicity Group"] == "WHITE_USA")
]["Headcount"].sum()

us_tech = data.loc[
    (data["Pay Group Country Desc"] == "United States of America")
    & (data["Technical Job Family"] == "Yes")
]["Headcount"].sum()

# total woman in Tech Global
ww_w_tech = data.loc[
    (data["Gender Code"] == "F")
    & (data["Technical Job Family"] == "Yes")
]["Headcount"].sum()

ww_tech = data.loc[
    data["Technical Job Family"] == "Yes"
]["Headcount"].sum()



# Percentage of US ethnic women are in leadership (Dir+) vs. white women
lead_eth_women_p = round((us_eth_lead / us_lead) * 100, 1).astype(str) + "%"
lead_white_women_p = round((us_white_lead / us_lead) * 100, 1).astype(str) + "%"

# Percentage of ethnic women in tech (US) vs white women
tech_eth_women_p = round((us_eth_tech / us_tech) * 100, 1).astype(str) + "%"
tech_white_women_p = round((us_white_tech / us_tech) * 100, 1).astype(str) + "%"

# Percentage of women in tech global
ww_tech_women = round((ww_w_tech / ww_tech) * 100, 1).astype(str) + "%"

# Percentage of women in lead global
ww_lead_women_p = round((ww_women_lead / ww_lead) * 100, 1).astype(str) + "%"


In [7]:
us_eth_lead

76

In [8]:
kpi = [
    "• Percentage of US ethnic women are in leadership (Dir+) vs. white women",
    "• Total woman in leadership US and global",
    "• Percentage of ethnic women in tech (US) vs white women",
    "• Percentage of women in tech global",
    "• Percentage of women in leadership global"
]

kpi_data = [
    "US Lead ethnic women: " + str(lead_eth_women_p) + ", US Lead white women: " + str(lead_white_women_p),
    "US Lead women: " + str(us_lead_w) + ", WW Lead Women: " + str(ww_women_lead),
    "US ethnic women tech: " + str(tech_eth_women_p) + ", US white women tech: " + str(tech_white_women_p),
    "WW women in tech: " + str(ww_tech_women),
    "WW women in leadership: " + str(ww_lead_women_p),
]

kpi_to_list = list(zip(kpi, kpi_data))

# Converting lists of tuples into
# pandas Dataframe.
df = pd.DataFrame(kpi_to_list, columns=["Metric", "Result"])

# Print data.
df


,Metric,Result
0,• Percentage of US ethnic women are in leaders...,"US Lead ethnic women: 11.1%, US Lead white wom..."
1,• Total woman in leadership US and global,"US Lead women: 225, WW Lead Women: 407"
2,• Percentage of ethnic women in tech (US) vs w...,"US ethnic women tech: 7.2%, US white women tec..."
3,• Percentage of women in tech global,WW women in tech: 22.9%
4,• Percentage of women in leadership global,WW women in leadership: 32.3%


In [9]:
ods.export_data(df,"Human_Capital_Report","Yes")